In [2]:
def convert_bstring_to_bin(bstring):
  bin_str = ""
  for char in bstring:
    bin_str += bin(ord(char))[2:] + ' '
  return bin_str


print(convert_bstring_to_bin('1234'))


110001 110010 110011 110100 
